# DD_FOM
Notebook to implement and test the DD formulation for the time-dependent 2D Burgers' equation.  
Author: Alejandro Diaz  
Date: 1/4/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.sparse as sp
from time import time
from utils.Burgers2D_probgen import Burgers2D
from utils.helpers import sp_diag
from utils.domain_decomposition import subdomain_indices, state_component, DD_FOM
from IPython.display import HTML, Image
from utils.solvers import newton_solve
import sys, os
import dill as pickle

In [2]:
plt.rc('font', size=20)
plt.rcParams['text.usetex'] = True

In [3]:
# make directories for figures and data
data_dir = './data/'
fig_dir0 = './figures/'
for d in [data_dir, fig_dir0]:
    if not os.path.exists(d): os.mkdir(d)

In [4]:
# parameters for physical domain and FD discretization
x_lim = [0, 1]
y_lim = [0, 1]
t_lim = [0, 2]
nx, ny, nt = 60, 60, 1500
viscosity = 1e-4

In [5]:
# initialize model
print('Initializing Burgers model...')
sys.stdout.flush()
fom = Burgers2D(nx, ny, nt, x_lim, y_lim, t_lim, viscosity)
print('Done!')

Initializing Burgers model...
Done!


In [6]:
# parameterized initial conditions
def u0(XY, mu):
    val = np.zeros(len(XY))
    for i, xy in enumerate(XY):
        if np.all([xy[0] >= 0.0, xy[0] <= 0.5, xy[1] >= 0.0, xy[1] <= 0.5]):
            val[i] = mu*np.sin(2*np.pi*xy[0])*np.sin(2*np.pi*xy[1])
    return val 

def v0(XY, mu):
    val = np.zeros(len(XY))
    for i, xy in enumerate(XY):
        if np.all([xy[0] >= 0.0, xy[0] <= 0.5, xy[1] >= 0.0, xy[1] <= 0.5]):
            val[i] = mu*np.sin(2*np.pi*xy[0])*np.sin(2*np.pi*xy[1])
    return val 

mu = 1.1

# set initial condition
fom.set_initial(lambda xy: u0(xy, mu), lambda xy: v0(xy, mu))

In [7]:
# load data
file = data_dir + f'nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/mu_{mu}_uv_state.p'
uv = pickle.load(open(file, 'rb'))
runtime0 = uv['runtime']
uv = uv['solution']

In [8]:
# # solve monolithic fom
# print('Solve Burgers equation with backward Euler:')
# sys.stdout.flush()
# uu, vv, res_hist, step_hist, runtime = fom.solve(tol=1e-9, maxit=20, print_hist=False)
# print('Done!') 
# sys.stdout.flush()

In [9]:
nsub_x, nsub_y = 2, 2
ddfom = DD_FOM(fom, nsub_x, nsub_y)
ddfom.set_initial(lambda xy: u0(xy, mu), lambda xy: v0(xy, mu))

In [10]:
uu = uv[:, :fom.nxy]
vv = uv[:, fom.nxy:]
u_intr, v_intr, u_intf, v_intf = [], [], [], []
dd_data_dir = data_dir + f'nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/DD_{nsub_x}x_by_{nsub_y}y/' 
for i, s in enumerate(ddfom.subdomain):
    sub_dir = dd_data_dir + f'sub_{i+1}of{ddfom.n_sub}/'
    uv_intr = pickle.load(open(sub_dir + f'interior/mu_{mu}_uv_state.p', 'rb'))
    runtime = uv_intr['runtime']
    uv_intr = uv_intr['solution']
    u_intr.append(uv_intr[:, :s.interior.indices.size])
    v_intr.append(uv_intr[:, s.interior.indices.size:])
    
    uv_intf = pickle.load(open(sub_dir + f'interface/mu_{mu}_uv_state.p', 'rb'))
    runtime = uv_intf['runtime']
    uv_intf = uv_intf['solution']
    u_intf.append(uv_intf[:, :s.interface.indices.size])
    v_intf.append(uv_intf[:, s.interface.indices.size:])

In [11]:
# u_full, v_full, u_intr, v_intr, u_intf, v_intf, lam, runtime = ddfom.solve(tol=1e-9, maxit=15)

In [12]:
abserr = np.zeros(ddfom.nt+1)
for i, s in enumerate(ddfom.subdomain):
    abserr += np.sum(np.square(uu[:, s.interior.indices]-u_intr[i]), 1) + \
              np.sum(np.square(vv[:, s.interior.indices]-v_intr[i]), 1) + \
              np.sum(np.square(uu[:, s.interface.indices]-u_intf[i]), 1) + \
              np.sum(np.square(vv[:, s.interface.indices]-v_intf[i]), 1)
    
abserr = np.max(np.sqrt(ddfom.hx*ddfom.hy*abserr/ddfom.n_sub))
print(f'DD FOM absolute error = {abserr:1.4e}')

DD FOM absolute error = 9.8773e-18


In [13]:
num = np.zeros(ddfom.nt+1)
den = np.zeros(ddfom.nt+1)
for i, s in enumerate(ddfom.subdomain):
    num += np.sum(np.square(uu[:, s.interior.indices]-u_intr[i]), 1) + \
           np.sum(np.square(vv[:, s.interior.indices]-v_intr[i]), 1) + \
           np.sum(np.square(uu[:, s.interface.indices]-u_intf[i]), 1) + \
           np.sum(np.square(vv[:, s.interface.indices]-v_intf[i]), 1)
    den += np.sum(np.square(uu[:, s.interior.indices]), 1) + \
           np.sum(np.square(vv[:, s.interior.indices]), 1) + \
           np.sum(np.square(uu[:, s.interface.indices]), 1) + \
           np.sum(np.square(vv[:, s.interface.indices]), 1)
relerr = np.max(np.sqrt(num))/np.max(np.sqrt(den))
print(f'DD FOM relative error = {relerr:1.4e}')

DD FOM relative error = 5.0786e-17
